In [1]:
import os
os.chdir(r"/Users/Greer/Desktop/Lending Club - JL")

##### IMPORT ONE CSV
import numpy as np
import pandas as pd
import seaborn as sns

## Open Up Original Date File
original file was imported using code in [LendingClub_import](LendingClub_import.ipynb)

In [ ]:
df = pd.read_csv("LendingClub-AllLoans.csv",low_memory=False)
df.reset_index
# 1218316 rows

In [9]:
df.drop(df.columns[[0]], axis=1, inplace=True)
# list attributes
list(df)

['id',
 'member_id',
 'loan_amnt',
 'funded_amnt',
 'funded_amnt_inv',
 'term',
 'int_rate',
 'installment',
 'grade',
 'sub_grade',
 'emp_title',
 'emp_length',
 'home_ownership',
 'annual_inc',
 'verification_status',
 'issue_d',
 'loan_status',
 'pymnt_plan',
 'url',
 'desc',
 'purpose',
 'title',
 'zip_code',
 'addr_state',
 'dti',
 'delinq_2yrs',
 'earliest_cr_line',
 'inq_last_6mths',
 'mths_since_last_delinq',
 'mths_since_last_record',
 'open_acc',
 'pub_rec',
 'revol_bal',
 'revol_util',
 'total_acc',
 'initial_list_status',
 'out_prncp',
 'out_prncp_inv',
 'total_pymnt',
 'total_pymnt_inv',
 'total_rec_prncp',
 'total_rec_int',
 'total_rec_late_fee',
 'recoveries',
 'collection_recovery_fee',
 'last_pymnt_d',
 'last_pymnt_amnt',
 'next_pymnt_d',
 'last_credit_pull_d',
 'collections_12_mths_ex_med',
 'mths_since_last_major_derog',
 'policy_code',
 'application_type',
 'annual_inc_joint',
 'dti_joint',
 'verification_status_joint',
 'acc_now_delinq',
 'tot_coll_amt',
 'tot_cur_

In [12]:
len(df)

1218316

In [11]:
df.head()

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,...,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit
0,1077501,1296599.0,5000.0,5000.0,4975.0,36 months,10.65%,162.87,B,B2,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN
1,1077430,1314167.0,2500.0,2500.0,2500.0,60 months,15.27%,59.83,C,C4,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN
2,1077175,1313524.0,2400.0,2400.0,2400.0,36 months,15.96%,84.33,C,C5,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN
3,1076863,1277178.0,10000.0,10000.0,10000.0,36 months,13.49%,339.31,C,C1,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN
4,1075358,1311748.0,3000.0,3000.0,3000.0,60 months,12.69%,67.79,B,B5,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN


In [13]:
##############################################################################
### work through our attributes slice-by-slice and get what we really need. 
### drop some useless attributes and clean-up/modify others. 
### work through the columns in batches to keep the cognitive burden low
##############################################################################

## For dataframes .iloc[row slice, column slice] 
df.iloc[:4,:7]
df.drop(['id','member_id'],1, inplace=True)

# 'int_rate' was loaded as an object data type instead of float due to the '%' character. 
# Let's strip that out and convert the column type.
df.int_rate = pd.Series(df.int_rate).str.replace('%', '').astype(float)

print (df.loan_amnt != df.funded_amnt).value_counts()
#there are no instances in the data where the requested amount doesn't match the funded

0          False
1          False
2          False
3          False
4          False
5          False
6          False
7          False
8          False
9          False
10         False
11         False
12         False
13         False
14         False
15         False
16         False
17         False
18         False
19         False
20         False
21         False
22         False
23         False
24         False
25         False
26         False
27         False
28         False
29         False
           ...  
1218286    False
1218287    False
1218288    False
1218289    False
1218290    False
1218291    False
1218292    False
1218293    False
1218294    False
1218295    False
1218296    False
1218297    False
1218298    False
1218299    False
1218300    False
1218301    False
1218302    False
1218303    False
1218304    False
1218305    False
1218306    False
1218307    False
1218308    False
1218309    False
1218310    False
1218311    False
1218312    False
1218313    Fal

AttributeError: 'NoneType' object has no attribute 'value_counts'

In [14]:
df.iloc[:5,8:15]

### Check out unique values of employment title, then drop
print(df.emp_title.value_counts().head())
print(df.emp_title.value_counts().tail())
df.drop(['emp_title'],1, inplace=True)

### Check out employment length counts, remove
df.emp_length.value_counts()
# convert to numerical data
df.replace('n/a', np.nan,inplace=True)
df.emp_length.fillna(value=0,inplace=True)
df['emp_length'].replace(to_replace='[^0-9]+', value='', inplace=True, regex=True)
df['emp_length'] = df['emp_length'].astype(int)

### Income verification status
df.verification_status.value_counts()

### Loan statuses ###
print(df.loan_status.value_counts())
issue_d_todate = pd.to_datetime(df.issue_d)
df.issue_d = pd.Series(df.issue_d).str.replace('-2014', '')
#issue_d_todate.value_counts().sort_index().plot(kind='bar')

Teacher             19848
Manager             16924
Owner                9087
Registered Nurse     8086
RN                   7864
Name: emp_title, dtype: int64
SAP BPC SME                             1
Lifetech                                1
Developmental Disability Consultants    1
one hour delivery                       1
Sungard Securitites Processing          1
Name: emp_title, dtype: int64
Current                                                708949
Fully Paid                                             379408
Charged Off                                             93270
Late (31-120 days)                                      20150
In Grace Period                                          8998
Late (16-30 days)                                        4699
Does not meet the credit policy. Status:Fully Paid       1988
Does not meet the credit policy. Status:Charged Off       761
Default                                                    78
Name: loan_status, dtype: int64


In [15]:
df.iloc[:5,12:21]

### Print purpose of loan
print(df.purpose.value_counts())
print(df.title.value_counts().tail())
df.drop(['pymnt_plan','url','desc','title' ],1, inplace=True)

debt_consolidation    713405
credit_card           277273
home_improvement       75617
other                  63983
major_purchase         25214
small_business         13952
car                    12515
medical                12512
moving                  7873
vacation                7169
house                   5235
wedding                 2348
renewable_energy         782
educational              423
Name: purpose, dtype: int64
seadoo racer       1
The end of debt    1
Windows project    1
Driveway Debt      1
homeimp            1
Name: title, dtype: int64


In [ ]:
df.iloc[:5,17:25]

### Examine the distribution of delinquency across all applicants
df.delinq_2yrs.value_counts()

### Convert earliest credit line to # years held credit line
from datetime import datetime
df.earliest_cr_line = pd.to_datetime(df.earliest_cr_line)
dttoday = datetime.now().strftime('%Y-%m-%d')
df.earliest_cr_line = df.earliest_cr_line.apply(lambda x: (
        np.timedelta64((x - pd.Timestamp(dttoday)),'D').astype(int))/-365)
df.earliest_cr_line
##*** DIDN'T WORK!!***##

### Create FICO range and average
df['fico_range'] = df.fico_range_low.astype('str') + '-' + df.fico_range_high.astype('str')
df['meanfico'] = (df.fico_range_low + df.fico_range_high)/2
df.drop(['fico_range_low','fico_range_high','initial_list_status'],1, inplace=True)

df['last_fico_range'] = df.last_fico_range_low.astype('str') + '-' + df.last_fico_range_high.astype('str')
df['last_meanfico'] = (df.last_fico_range_low + df.last_fico_range_high)/2
df.drop(['last_fico_range_high','last_fico_range_low','policy_code'],1, inplace=True)
##*** ATTRIBUTE DOES NOT EXIST?? ***###

In [16]:
df.iloc[:5,23:32]

### Convert revolving utility to integer, no %
df.revol_util = pd.Series(df.revol_util).str.replace('%', '').astype(float)
df.drop(['initial_list_status'],1, inplace=True)

### Fill in N/A with 0
df.fillna(0.0,inplace=True)
df.fillna(0,inplace=True)

df.info()

### In the above, we have addressed missing data, converted strings to 
# numerical representations where possible, dropped superfluous attributes

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1218316 entries, 0 to 1218315
Columns: 103 entries, loan_amnt to total_il_high_credit_limit
dtypes: float64(86), int64(1), object(16)
memory usage: 957.4+ MB


In [ ]:
## Save cleaned dataset to .csv
df.to_csv("Lending Club - All Loans - clean.csv")